In [1]:
#Install

!pip install langchain --quiet
!pip install openai --quiet
!apt-get install -y libpoppler-cpp-dev --quiet
!pip install PyMuPDF --quiet
!pip install -U gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 50.5 kB of archives.
After this op

In [2]:
# Import AI stuff.
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage


# Graido for UI, fitz for pdf reading and generation
import gradio as gr
import fitz

import textwrap


# Create llm.
llm = ChatOpenAI(openai_api_key="API_KEY",
                 model="gpt-3.5-turbo-0613")



#Define RewriteBot. It takes three inputs.
def RewriteBot(inputpdf, inputtext, complexity_level):

    # Creating empty string
    pdftext = ""

    # Adding pdf text to string
    pdftext += extract_text_from_pdf(inputpdf)

    # Adding optional input text to string
    pdftext += inputtext

    # Adding the text to the AI model
    history_langchain_format = [
        HumanMessage(content=pdftext),
    ]



    return text_to_pdf(inputtext, complexity_level)


    # Appending AI instructions depending on the selected complexity level

    if complexity_level == "Slang":
        history_langchain_format.append(SystemMessage(content="Rewrite this classic text in a modern way, using a lot of slang. Don't shorten the text:"))

    if complexity_level == "Informal":
        history_langchain_format.append(SystemMessage(content="Rewrite this in more modern informal language. Don't shorten the text:"))

    if complexity_level == "Semi-formal":
        history_langchain_format.append(SystemMessage(content="Rewrite this in more modern semi-formal language. Don't shorten the text:"))

    if complexity_level == "Formal":
            history_langchain_format.append(SystemMessage(content="Rewrite this classic text in a modern, easy to understand, formal way. Don't shorten the text:"))

    # Sending request to gpt
    gpt_response = llm(history_langchain_format)

    # Converting gpt response to pdf
    pdffile = text_to_pdf(gpt_response.content, complexity_level)

    # Returning pdf file, original text from pdf, and the rewritten text. The order depends on which order it is shown in the UI
    return pdffile, pdftext, gpt_response.content


def extract_text_from_pdf(pdf_file_bytes):
    # Attempt to extract text. If error, return empty string
    try:
        # Create a temporary file to write the byte stream
        with open("/tmp/uploaded_pdf.pdf", "wb") as temp_file:
            temp_file.write(pdf_file_bytes)

        # Open the temporary file with fitz
        pdf_document = fitz.open(temp_file.name)
        text = ""

        # Goes through each page and adds text to string
        for page_num in range(pdf_document.page_count):
            page = pdf_document[page_num]
            page_text = page.get_text()
            page_text = page_text.replace("-\n", "")  # Remove hyphen followed by newline
            text += page_text

        return text
    # If error, return error message
    except Exception as e:
        return ""


def text_to_pdf(text, complexity_level):
    # Attempt to generate pdf, if error return empty string
    try:
        # Replace non-standard quotes with standard double quotes
        text = text.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

        # Create a fitz Document
        pdf_document = fitz.open()

        # Define page dimensions
        width, height = 600, 700

        # Define text properties like font and font size
        fontname, fontsize = "tiro", 16
        text_width = width - 1  # Adjust as needed

        # Add a new page
        page = pdf_document.new_page(width=width, height=height)

        # Define margins
        left_margin, top_margin, bottom_margin = 50, 70, 70

        # Starting coordinates for text
        x, y = left_margin, top_margin

        # Split the text into lines based on width while keeping original line breaks
        lines = []
        for paragraph in text.split('\n'):
            lines.extend(textwrap.wrap(paragraph, width=int(text_width / (0.5 * fontsize))))

        rect = fitz.Rect(left_margin, top_margin, width - left_margin, height - bottom_margin)
        shape = page.new_shape()
        shape.draw_rect(rect)
        shape.finish(width = 0.3, color = (1,0,0), fill = (1,1,0))
        shape.insert_textbox(rect, lines, align=3)
        shape.commit()

        # Iterate through lines, adding the text to the PDF line by line
#        for line in lines:
            # Check if text goes beyond page bounds
#            if y + fontsize + 5 > height + top_margin - bottom_margin:
                # Add a new page and reset coordinates
#                page = pdf_document.new_page(width=width, height=height)

            # Insert text and increment coordinates
#            y += fontsize + 5  # Increment y-coordinate. Adjust as needed

        # Save the PDF to a file
        pdf_filename = complexity_level + ".pdf"
        pdf_document.save(pdf_filename)
        return pdf_filename
    except Exception as e:
        return ""

#Create the interface with Gradio and put labels and descriptions.
demo = gr.Interface(
    # Defining which function to use with the input and outputs
    fn=RewriteBot,
    inputs=[
        gr.File(label="Upload PDF", type="binary"),
        gr.Textbox(label="Or input text manually"),
        gr.Radio(["Slang", "Informal", "Semi-formal", "Formal"], label="Level of formality", info="", value="Semi-formal"),
    ],
    # Outputs are returned from the function RewriteBot
    outputs= [
        gr.File(label="PDF Output"),
        #gr.Textbox(label="Original text"),
        #gr.Textbox(label="Rewritten text"),
    ],

    live="True",
    title="Book Rewriter",
    description="")
demo.launch(share=True)

ModuleNotFoundError: No module named 'langchain_community'